In [2]:
import requests  
import json  
import time  
  
# Azure Cognitive Services 엔드포인트와 API 키  
# endpoint = "YOUR_ENDPOINT_HERE"  
# api_key = "YOUR_KEY_HERE"  
# api_version = '2022-10-01-preview'  
  
# 텍스트 분류 작업 제출 URL  
url = f"{endpoint}/language/analyze-text/jobs?api-version={api_version}"  
  
# 요청 헤더  
headers = {  
    "Ocp-Apim-Subscription-Key": api_key,  
    "Content-Type": "application/json"  
}  
  
# 요청 데이터 (예시 데이터)  
data = {  
    "tasks": [  
        {  
            "kind": "CustomEntityRecognition",  
            "parameters": {  
                "projectName": "customnerdemo",  
                "deploymentName": "cnerdemodeploy",  
                "stringIndexType": "TextElement_v8"  
            }  
        }  
    ],  
    "displayName": "CustomTextPortal_CustomEntityRecognition",  
    "analysisInput": {  
        "documents": [  
            {  
                "id": "document_CustomEntityRecognition",  
                "text": "Casey Jensen with a mailing address of 2469 Pennsylvania Avenue, City of New Brunswick, State of New Jersey",  
                "language": "en"  
            }  
        ]  
    }  
}  
  
# 텍스트 분류 작업 제출  
response = requests.post(url, headers=headers, data=json.dumps(data))  
  
# 응답 확인  
print(f"Response status code: {response.status_code}")  
  
# 응답 본문 출력 (디버깅용)  
print("Response text:")  
print(response.text)  
  
# 응답 처리  
if response.status_code == 202:  # Accepted 상태일 때만 처리  
    operation_location = response.headers.get("Operation-Location")  
    if operation_location:  
        print(f"Operation location: {operation_location}")  
  
        # 비동기 작업의 결과를 가져오기 위해 주기적으로 상태 확인  
        while True:  
            result_response = requests.get(operation_location, headers={"Ocp-Apim-Subscription-Key": api_key})  
            if result_response.status_code == 200:  
                result = result_response.json()  
                status = result.get("status")  
  
                if status == "succeeded":  
                    print("Analysis result:")  
                    #print(json.dumps(result, indent=4)) # 모두 출력
                    # documents 부분만 출력  
                    tasks = result.get("tasks", {})  
                    items = tasks.get("items", [])  
                    if items:  
                        documents = items[0].get("results", {}).get("documents", [])  
                        print("Documents:")  
                        print(json.dumps(documents, indent=4))  
                    else:  
                        print("No items found in the tasks.")   
                    break  
                elif status in ("failed", "notStarted"):  
                    print(f"Analysis status: {status}")  
                    print("Waiting for 5 seconds before retrying...")  
                    time.sleep(5)  # 5초 대기 후 다시 요청  
                else:  
                    print(f"Analysis status: {status}")  
                    print("Waiting for 5 seconds before retrying...")  
                    time.sleep(5)  # 5초 대기 후 다시 요청  
            else:  
                print(f"Error: {result_response.status_code}")  
                print(result_response.text)  
                break  
    else:  
        print("Error: 'Operation-Location' header not found in the response.")  
else:  
    print(f"Error: {response.status_code}")  
    print(response.text)  


Response status code: 202
Response text:

Operation location: https://chrlin32lang.cognitiveservices.azure.com/language/analyze-text/jobs/edcfa033-fa4c-4443-83d7-17ffd4a3eb9c?api-version=2022-10-01-preview
Analysis status: notStarted
Waiting for 5 seconds before retrying...
Analysis result:
Documents:
[
    {
        "id": "document_CustomEntityRecognition",
        "entities": [
            {
                "text": "Casey Jensen",
                "category": "BorrowerName",
                "offset": 0,
                "length": 12,
                "confidenceScore": 0.99
            },
            {
                "text": "2469 Pennsylvania Avenue",
                "category": "BorrowerAddress",
                "offset": 39,
                "length": 24,
                "confidenceScore": 0.87
            },
            {
                "text": "New Brunswick",
                "category": "BorrowerCity",
                "offset": 73,
                "length": 13,
                "c